# Scraping exercise - Top 100 DJ Mag

Este script objetiva coletar a lista de Top 100 de melhores DJs de cada ano desde de 2004, da revista renomada DJ Mag, e armazenar em um dataset no formato dataframe do pandas.

In [12]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import re

#testando a url
url = 'https://djmag.com/top100dj?year=2020'

#A biblioteca requests é utilziada para solicitações e comunicações http de forma organizada, facilitando o processo de scraping
response = get(url)
response

<Response [200]>

Já a biblioteca BeautifulSoup atua como facilitadora no processo de scraping de páginas em HTML, permitindo a interação do python com o código da página clusterizado em parâmetros definidos.


In [13]:
djmag_soup = BeautifulSoup(response.text)
djmag_soup

<!DOCTYPE html>
<!--[if IEMobile 7]><html xmlns:wb="http://open.weibo.com/wb" class="no-js ie iem7" lang="en" dir="ltr"><![endif]--><!--[if lte IE 6]><html xmlns:wb="http://open.weibo.com/wb" class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]--><!--[if (IE 7)&(!IEMobile)]><html xmlns:wb="http://open.weibo.com/wb" class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]--><!--[if IE 8]><html xmlns:wb="http://open.weibo.com/wb" class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]--><!--[if (gte IE 9)|(gt IEMobile 7)]><html xmlns:wb="http://open.weibo.com/wb" class="no-js ie" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]--><!--[if !IE]><!--><!--<html xmlns:wb="http://open.weibo.com/wb" class="no-js" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp

A função abaixo serve para encontrar os containers dentro do código da página HTML que contém os parâmetros que desejmaos capturar para a composição do dataset.

In [14]:
djmag_containers = djmag_soup.find_all('div', class_ = 'container_with_line')
djmag_containers

[<div class="container_with_line">
 <article>
 <div class="views-field views-field-field-media"> <div class="field-content"><a href="/top-100-djs/poll-2020-david-guetta"><div class="media" id="file-49473">
 <figure class="media--social_media_image">
 <img alt="" class="media--social_media_image--social_media_image" src="https://djmag.com/sites/default/files/styles/social_media_image/public/top100/djs/image/David%20Guetta%20Dj%20Mag1098.jpg?itok=2Qn1rjN-"/> <figcaption class="media--social_media_image--caption">
 </figcaption>
 </figure>
 </div>
 </a></div> </div>
 <div class="views-field views-field-field-top100-position"> <div class="field-content">1</div> </div>
 <div class="views-field views-field-field-top100-places"> <div class="field-content"><div class="top100dj-title-bar">
 <div class="top100dj-position">
     1
   </div>
 <div class="top100dj-name">
 <a href="/top-100-djs/poll-2020-david-guetta">David Guetta</a>
 </div>
 <div class="top100dj-movement">
     Up 2
   </div>
 </d

A linha de código abaixo faz um teste com o primeiro container obtido após a utilização da função find_all() do BeautifulSoup, e permite a leitura dos atributo que serão buscados para a captação e armazenamento posterior no dataset.

Dessa forma, busca-se os atributos relacionados à posição do DJ na lista, o nome do DJ e o movimento em relação ao ano anterior.

In [15]:
djmag_containers[0]

<div class="container_with_line">
<article>
<div class="views-field views-field-field-media"> <div class="field-content"><a href="/top-100-djs/poll-2020-david-guetta"><div class="media" id="file-49473">
<figure class="media--social_media_image">
<img alt="" class="media--social_media_image--social_media_image" src="https://djmag.com/sites/default/files/styles/social_media_image/public/top100/djs/image/David%20Guetta%20Dj%20Mag1098.jpg?itok=2Qn1rjN-"/> <figcaption class="media--social_media_image--caption">
</figcaption>
</figure>
</div>
</a></div> </div>
<div class="views-field views-field-field-top100-position"> <div class="field-content">1</div> </div>
<div class="views-field views-field-field-top100-places"> <div class="field-content"><div class="top100dj-title-bar">
<div class="top100dj-position">
    1
  </div>
<div class="top100dj-name">
<a href="/top-100-djs/poll-2020-david-guetta">David Guetta</a>
</div>
<div class="top100dj-movement">
    Up 2
  </div>
</div>
</div> </div> </a

In [16]:
#testando o artibuto que contém a posição do DJ na lista

x = djmag_containers[0].find('div', class_='views-field views-field-field-top100-position').text
xint = re.findall(r'\d+',x)
int(xint[0])

1

In [17]:
#testando o atributo que contém o nome do DJ

djmag_containers[0].find('div', class_='top100dj-name').a.text

'David Guetta'

In [18]:
#testando o atributo que resgata o movimento do DJ na lista, em comparação ao ano anterior

test = djmag_containers[0].find('div', class_='top100dj-movement').text
test_pattern = re.sub('[\r\n ]', '', test)
test_pattern

'Up2'

Tendo encontrado os atributos que possuem as informações que busca-se no site da DJ Mag, podemos construir o código para iterar as buscas de acordo com o ano referência do Top 1000 que desejamos buscar, e coletar os dados da lista completa.

In [19]:
#Definindo o range de tempo base para coleta dos dados. No caso, caputramos de 2004 a 2020.
years = [str(i) for i in range(2004,2021)]

#criando o código que irá iterar pela lista de anos e coletará os dados
dj_mag_year = []
dj_mag_pos = []
dj_mag_djname = []
dj_mag_mov = []

for y in years:
    url = 'https://djmag.com/top100dj?year={}'.format(y)
    resp = get(url)
    
    djmag_soup = BeautifulSoup(response.text)
    
    djmag_containers = djmag_soup.find_all('div', class_ = 'container_with_line')
    
    for i in range(0,len(djmag_containers)):
        #armazeando o ano referênncia
        dj_mag_year.append(str(y))
        
        #armazenando a posição do DJ
        pos = djmag_containers[i].find('div', class_='views-field views-field-field-top100-position').text
        pos_num = re.findall(r'\d+',pos)
        dj_mag_pos.append(pos_num[0])
        
        #armazenando o nome do DJ
        djname = djmag_containers[i].find('div', class_='top100dj-name').a.text
        dj_mag_djname.append(djname)
        
        #armazenando a movimentação do DJ vs ano anterior
        mov = djmag_containers[i].find('div', class_='top100dj-movement').text
        mov_str_pattern = re.sub('[\r\n ]', '', mov)
        dj_mag_mov.append(mov_str_pattern)

In [20]:
#Criando o dataframe com os dados coletados
df_djmag_top100 = pd.DataFrame({'year':dj_mag_year,
                                'ranking':dj_mag_pos,
                                'dj_name':dj_mag_djname,
                                'movement':dj_mag_mov})
df_djmag_top100

,year,ranking,dj_name,movement
0,2004,1,David Guetta,Up2
1,2004,2,Dimitri Vegas & Like Mike,Down1
2,2004,3,Martin Garrix,Down1
3,2004,4,Armin Van Buuren,NonMover
4,2004,5,Alok,Up6
...,...,...,...,...
1695,2020,96,Dimitri Vangelis & Wyman,NewEntry
1696,2020,97,Blastoyz,NewEntry
1697,2020,98,22Bullets,NewEntry
1698,2020,99,Robin Schulz,Down17


Tendo finalizado o processo de web scraping, o dataframe pode ser tanto armazenado em um banco de dados SQL, ou como desejar extrair as informações para serem armazenadas e trabalhadas. Como o objetivo do script é de coletar os dados, não vou abordar neste código as técnicas de extração.